# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
# Inicializando Spark
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("linearRegHyu").getOrCreate()

22/05/11 14:05:11 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/11 14:05:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/11 14:05:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/11 14:05:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/11 14:05:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/11 14:05:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
# Imports
# Regressão linear
from pyspark.ml.regression import LinearRegression

# Vetores
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Funções
from pyspark.ml.feature import StringIndexer

In [13]:
# Load data
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
# Viz
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [14]:
# Stringindexer - Ajustando tipo da Cruise_line

# Criando nova coluna com Strinindexer
label_stringIdx = StringIndexer(inputCol = 'Cruise_line', outputCol="cruise_line_index")
label_stringIdx_model = label_stringIdx.fit(data)
data = label_stringIdx_model.transform(data)

# Visualizando Schema
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_line_index: double (nullable = false)



In [15]:
# Viz
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_line_index=16.0)]

* Alvo: **Crew** - quantidade de crew members necessária

* Features: variar de acordo com os modelos

In [16]:
# Colunas
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_line_index']

In [18]:
# Modelo 1

# Criando as features (características)
assembler_1 = VectorAssembler(inputCols = ['Age',
                                           'Tonnage',
                                           'passengers',
                                           'length',
                                           'cabins',
                                           'passenger_density',
                                           'cruise_line_index'],
                             outputCol='features')


# Transformando os dados
output_1 = assembler_1.transform(data)

output_1.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_line_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [69]:
# Modelo 2 - Sem passengers density

# Criando as features (características)
assembler_2 = VectorAssembler(inputCols = ['Age',
                                           'Tonnage',
                                           'passengers',
                                           'length',
                                           'cabins',
                                           'cruise_line_index'],
                             outputCol='features')


# Transformando os dados
output_2 = assembler_2.transform(data)

output_2.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_line_index: double (nullable = false)
 |-- features: vector (nullable = true)



## Modelo 1

In [70]:
# Selecionando Dados finais
final_data_1 = output_1.select('features', 'crew')

# Separando entre test e treino
train_data_1, test_data_1  = final_data_1.randomSplit([0.7,0.3])

# Visualizando Treino
train_data_1.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,86.0,21.04,9...|  8.0|
|[5.0,122.0,28.5,1...|  6.7|
|[5.0,133.5,39.59,...|13.13|
|[6.0,30.276999999...| 3.55|
|[6.0,30.276999999...| 3.55|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,93.0,23.94,9...|11.09|
|[6.0,110.23899999...| 11.5|
|[6.0,112.0,38.0,9...| 10.9|
|[6.0,113.0,37.82,...| 12.0|
|[6.0,158.0,43.7,1...| 13.6|
|[7.0,89.6,25.5,9....| 9.87|
|[8.0,91.0,22.44,9...| 11.0|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,59.058,17.0,...|  7.4|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,85.0,19.68,9...| 8.69|
|[9.0,90.09,25.01,...| 8.69|
|[9.0,113.0,26.74,...|12.38|
+--------------------+-----+
only showing top 20 rows



In [71]:
# Descrevendo treino 
train_data_1.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               113|
|   mean| 7.651681415929214|
| stddev|3.5560757621526307|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [73]:
# Descrevendo teste
test_data_1.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               45|
|   mean|8.152000000000003|
| stddev|3.380128830250543|
|    min|             0.88|
|    max|             13.6|
+-------+-----------------+



In [74]:
# Estimando modelo
lr_1 = LinearRegression(labelCol= 'crew', regParam=0.1)

# Fitting modelo 1 aos dados treinados
lr_1_Model = lr_1.fit(train_data_1)

In [75]:
# Coeficientes
lr_1_Model.coefficients

DenseVector([-0.0015, 0.0163, 0.0, 0.3612, 0.5069, 0.0057, 0.0294])

In [76]:
# Intercepto
lr_1_Model.intercept

-1.116080128460442

### Avaliando o modelo 1

In [77]:
# Checando ajuste
test_results_1 = lr_1_Model.evaluate(test_data_1)

In [78]:
# R2
test_results_1.r2

0.9095714407396988

In [79]:
# RMSE
test_results_1.rootMeanSquaredError

1.0050927502649227

## Modelo 2

In [80]:
# Selecionando Dados finais
final_data_2 = output_2.select('features', 'crew')

# Separando entre test e treino
train_data_2, test_data_2  = final_data_2.randomSplit([0.7,0.3])

# Visualizando Treino
train_data_2.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[5.0,115.0,35.74,...| 12.2|
|[5.0,122.0,28.5,1...|  6.7|
|[5.0,133.5,39.59,...|13.13|
|[5.0,160.0,36.34,...| 13.6|
|[6.0,30.276999999...| 3.55|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,93.0,23.94,9...|11.09|
|[6.0,112.0,38.0,9...| 10.9|
|[6.0,113.0,37.82,...| 12.0|
|[7.0,89.6,25.5,9....| 9.87|
|[7.0,116.0,31.0,9...| 12.0|
|[8.0,77.499,19.5,...|  9.0|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,59.058,17.0,...|  7.4|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,85.0,19.68,9...| 8.69|
|[9.0,88.5,21.24,9...| 10.3|
|[9.0,105.0,27.2,8...|10.68|
|[9.0,110.0,29.74,...| 11.6|
|[9.0,113.0,26.74,...|12.38|
+--------------------+-----+
only showing top 20 rows



In [81]:
train_data_2.head(1)

[Row(features=DenseVector([5.0, 115.0, 35.74, 9.0, 15.32, 8.0]), crew=12.2)]

In [82]:
# Descrevendo treino 
train_data_2.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.824912280701764|
| stddev|3.3622319929680793|
|    min|              0.88|
|    max|              19.1|
+-------+------------------+



In [83]:
# Descrevendo teste
train_data_2.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.824912280701764|
| stddev|3.3622319929680793|
|    min|              0.88|
|    max|              19.1|
+-------+------------------+



In [84]:
# Estimando modelo
lr_2 = LinearRegression(labelCol= 'crew', regParam=0.1)

# Fitting modelo 1 aos dados treinados
lr_2_Model = lr_2.fit(train_data_2)

In [85]:
# Coeficientes
lr_2_Model.coefficients

DenseVector([-0.014, 0.0139, -0.0125, 0.4712, 0.495, 0.0302])

In [86]:
# Intercepto
lr_2_Model.intercept

-1.0957392385721814

### Avaliando o modelo 2

In [87]:
# Checando ajuste
test_results_2 = lr_2_Model.evaluate(test_data_2)

In [88]:
# R2
test_results_2.r2

0.9648011848156912

In [89]:
# RMSE
test_results_2.rootMeanSquaredError

0.7206967942520611

## Considerações Parciais

O modelo 2 performou melhor que o modelo 1, possivelmente porque há algum tipo de multicolinearidade entre as features passengers e passenger density. Ao eliminar esse probelma (modelo 2), o RMSE diminui e a variância do modelo é melhor controlada. 

No modelo 2, 

    * Idade do navio (age) afeta negativamente a quantidade de membros da tripulação
    
    * Peso do navio (Tonnage) afeta positivamente o tamanho da tripulação
    
    * Contraintuitivamente, o número de passageiros afeta negativamente
    
    * Tamanho e número de cabines são os elementos que mais afetam a quantidade de tripulação
    
    * A Cruise_Line afeta positivamente a quantidade de membros
    

### Deploying modelo 2

In [90]:
unlabeled_data = test_data_2.select('features')

unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,86.0,21.04,9...|
|[6.0,30.276999999...|
|[6.0,110.23899999...|
|[6.0,158.0,43.7,1...|
|[7.0,158.0,43.7,1...|
|[8.0,91.0,22.44,9...|
|[9.0,90.09,25.01,...|
|[9.0,113.0,26.74,...|
|[10.0,105.0,27.2,...|
|[11.0,58.6,15.66,...|
|[11.0,85.0,18.48,...|
|[11.0,91.0,20.32,...|
|[12.0,2.329,0.94,...|
|[12.0,138.0,31.14...|
|[13.0,30.27699999...|
|[13.0,76.0,18.74,...|
|[13.0,85.619,21.1...|
|[15.0,30.27699999...|
|[15.0,78.491,24.3...|
+--------------------+
only showing top 20 rows



In [91]:
predictions = lr_2_Model.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|20.169174811214795|
|[5.0,86.0,21.04,9...| 9.327653797931486|
|[6.0,30.276999999...| 4.193346134752166|
|[6.0,110.23899999...|11.763603111109987|
|[6.0,158.0,43.7,1...|14.683462235160107|
|[7.0,158.0,43.7,1...|14.608162354040156|
|[8.0,91.0,22.44,9...| 9.999425271851196|
|[9.0,90.09,25.01,...| 9.667222852395055|
|[9.0,113.0,26.74,...|11.175714122528445|
|[10.0,105.0,27.2,...|10.941733382489584|
|[11.0,58.6,15.66,...| 7.334520479859116|
|[11.0,85.0,18.48,...| 8.847028707847466|
|[11.0,91.0,20.32,...| 9.316413666442642|
|[12.0,2.329,0.94,...|0.5548958447515182|
|[12.0,138.0,31.14...|12.780254576414164|
|[13.0,30.27699999...|3.9414630587545245|
|[13.0,76.0,18.74,...| 8.609263311316148|
|[13.0,85.619,21.1...|  9.53634496340378|
|[15.0,30.27699999...| 3.913377394673378|
|[15.0,78.491,24.3...| 8.743064160302161|
+--------------------+------------

## Teste de realidade

In [92]:
from pyspark.sql.functions import corr # Correlação de Pearson

In [94]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [114]:
data.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [115]:
data.select(corr('crew', 'length')).show()

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+

